In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


#coding=utf-8
import time
import torch
import torchvision
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import torch.nn as nn
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_curve, roc_auc_score,precision_recall_curve, f1_score, recall_score, accuracy_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/telecom-churn-datasets/churn-bigml-80.csv')
df.info()

In [ ]:
# print(df.dtypes)
df.describe()

In [ ]:
df[['State', 'International plan','Voice mail plan']]

In [ ]:
corrmat = df.corr()
f, ax = plt.subplots(figsize=(20, 9))
sns.heatmap(corrmat, cmap='ocean', 
            vmax=0.8, 
            annot_kws={"size":5}, 
            square=True, annot = True)
plt.show()

In [ ]:
plt.figure(figsize=[5,5])
plt.title('Customer Attrition')
df['Churn'].value_counts().plot.pie(autopct='%1.2f%%')
plt.show()

In [ ]:
def preprocess_train(data_path):
    # load data
    df = pd.read_csv(data_path)
    for i in ['State', 'International plan','Voice mail plan','Churn']:
        df[i], unique = pd.factorize(df[i])

    df_01 = df.sort_values(by=['Churn'],ascending=True)
    df_balanced_1 = df_01[:388][:]
#     df_balanced_2 = df_01[388:][:].sample(388)
    df_balanced_2 = df_01[388:][:]
    df_balanced = pd.concat([df_balanced_1, df_balanced_2])
    
    df_value = np.array(df_balanced)
    scaler = StandardScaler()
    X = scaler.fit_transform(df_value[:,:19])
    Y = df_value[:,19].reshape(-1, 1)

    return X, Y

In [ ]:
def preprocess_test(data_path):
    # 1-1 load test data
    df = pd.read_csv(data_path)
    for i in ['State', 'International plan','Voice mail plan','Churn']:
        df[i], unique = pd.factorize(df[i])
    df_value = np.array(df)
    scaler = StandardScaler()
    X = scaler.fit_transform(df_value[:,:19])
    Y = df_value[:,19].reshape(-1, 1)
    
    return X, Y

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features=19, out_features=100), nn.Sigmoid(),
            nn.Linear(100, 128), nn.Sigmoid(), nn.BatchNorm1d(128), nn.Dropout(0.1),
            nn.Linear(128, 108), nn.Sigmoid(), nn.BatchNorm1d(108), nn.Dropout(0.1),
            nn.Linear(108, 100), nn.Sigmoid(), nn.BatchNorm1d(100), nn.Dropout(0.1),
            nn.Linear(100, 2), nn.Softmax(dim=1)
        )

    def forward(self, input:torch.FloatTensor):
        return self.net(input)


In [ ]:
epoch_number = 300
learning_rate = 0.1
threshold = 0.5

loss_all = []        # Collect all MSE loss
accuracy_all = []    # Collect all accuracy
acc_high = [0]      # Collect the highest accuracy
f1_all = []
recall_all = []

In [ ]:
X, Y = preprocess_train('../input/telecom-churn-datasets/churn-bigml-80.csv')
X_test, Y_test = preprocess_test('../input/telecom-churn-datasets/churn-bigml-20.csv')

X_tens = torch.tensor(X_test, dtype=torch.float)
Y_tens = torch.tensor(Y_test, dtype=torch.float)

data_set = TensorDataset(torch.tensor(X, dtype=torch.float),
                         torch.tensor(Y, dtype=torch.float))

# Create data_set of tensors with torch.utils.data.TensorDataset
dataloader = DataLoader(data_set,
                        batch_size=256,  # size = 16/64/128/256
                        shuffle=True)
net = Net().to(device)

# Set optimizer and loss function
# optim = torch.optim.SGD(Net.parameters(net), lr=0.05, momentum = 0.9)
optim = torch.optim.Adam(Net.parameters(net), lr=learning_rate, 
                         weight_decay=0.00001
                        )
# optim = torch.optim.RMSprop(Net.parameters(net),lr=learning_rate
#                             ,weight_decay=0.0001
#                             ,momentum=0.9
#                            )
criterion = nn.CrossEntropyLoss()


In [ ]:
def adjust_learning_rate(optimizer, epoch, lr):
    """lr decayed by __ every __ epochs"""
    lr *= (0.7 ** (epoch // 40))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

x_opt = list(range(epoch_number))
y_opt = []
lr_init = optim.param_groups[0]['lr']

for epoch in range(epoch_number):
    adjust_learning_rate(optim, epoch, lr_init)
    lr = optim.param_groups[0]['lr']
    print(epoch, lr)
    y_opt.append(lr)

In [ ]:
for epoch in range(epoch_number):
    loss = None
    for batch_x, batch_y in dataloader:
        y_predict = net(batch_x.to(device))
        loss = criterion(y_predict, batch_y.to(device).long().squeeze())
        
        optim.zero_grad()
        loss.backward()
        optim.step()

    # Print steps and loss every 100 loops
    if (epoch+1) % 3 == 0:
        predict = net(X_tens.to(device))
        loss = np.array(loss.detach().cpu().numpy())
        loss_all.append(loss)
        pred = torch.max(predict,dim=1)[1]
        # Print training informations
        accuracy = accuracy_score(pred.detach().cpu().numpy(), Y_test) * 100
        accuracy_all.append(accuracy)
        recl = recall_score(Y_test, pred.detach().cpu().numpy())
        f1_temp = f1_score(Y_test, pred.detach().cpu().numpy())
        recall_all.append(recl)
        f1_all.append(f1_temp)
        print("Step: {0}   Cross Entropy Loss: {1}   Accuracy: {2} % \nF1 score is {3} Recall score is {4}\n"
              .format(epoch+1, loss.item(),accuracy, f1_temp, recl))
        
        if recl*100 > acc_high[len(acc_high)-1]:
            acc_high.append(recl*100)    # Collect the highest accuracy rate currently
            pred_final = pred
            net_final = net
        else:
            pass

# # =====Draw Plots====================================================================================
#         fig, axs = plt.subplots(2,1,figsize=(12,10))
#         ax1, ax2 = axs.flatten()
#         x = np.linspace(0, len(predict), len(predict))
#         ax1.set(
# #                 ylim=[-0.4, 1.4],
#                 title='Recall')
# #         ax1.set_xlabel('Iteration times/100') # x label
#         ax1.set_ylabel('Recall value') # y label
#         ax1.plot(recall_all,c='r', label="Recall")
# #         ax1.scatter(x, Y_test,c='b', label="Test label")
#         ax1.legend()
# #         ax1.tick_params(axis='y', labelcolor='red')
        
#         ax2 = plt.gca()
#         ax2.set_ylabel('Cross Entropy Loss') # y label
#         ax2.plot(loss_all, c='b', label="Cross Entropy Loss")
#         ax2.tick_params(axis='y', labelcolor='blue')
#         ax3 = ax2.twinx()
#         ax3.set_ylabel('Accuracy(%)')
#         ax3.tick_params(axis='y', labelcolor='red')
#         ax3.plot(accuracy_all, c='r', label='Accuracy')

# #         plt.savefig(fname="./picture/" + str(epoch+1) + ".png", figsize=[10, 10])
#         plt.show()
# # =====================================================================================================

In [ ]:
print('Recall is %.3f' %(acc_high[-1]))

In [ ]:
# # # Test code
predict = net_final(X_tens.to(device))
pred = torch.max(predict,dim=1)[1]
pred = pred.detach().cpu().numpy()
predict = predict.detach().cpu().numpy()

test_pred = np.zeros([len(predict),1])
test_pred = predict[:,1]

fig, axs = plt.subplots(1,2,figsize=(12,5))
ax1, ax2 = axs.flatten()

precision, recall, thresholds = precision_recall_curve(Y_test, test_pred)
FP_rate, TP_rate, thresholds = roc_curve(Y_test, test_pred)
auc_value = roc_auc_score(Y_test, test_pred)
recall_final = recall_score(Y_test, pred_final.detach().cpu().numpy())
f1_value = f1_score(Y_test, pred_final.detach().cpu().numpy())
accur_final = accuracy_score(Y_test, pred_final.detach().cpu().numpy()) * 100

ax1.set(
#         xlim=[0, 1], ylim=[0, 1],
        title='P-R Curve (Precision & Recall)')
ax1.set_xlabel('Recall') # x label
ax1.set_ylabel('Precision') # y label
ax1.plot(recall, precision,c='b')

ax2.set(
#         xlim=[0, 1], ylim=[0, 1],
        title='ROC Curve')
ax2.set_xlabel('False positive rate') # x label
ax2.set_ylabel('True Positive rate') # y label
ax2.plot(FP_rate, TP_rate,c='b')

plt.show()

print('AUC value is {0} \nRecall value is {1} \nF1 value is {2}\nAccuracy is {3} %'
      .format(auc_value, recall_final, f1_value, accur_final))

In [ ]:
for parameters in net_final.parameters():
    print(parameters)

In [ ]:
# torch.save(net_final,'./model/bpnet_model.pt')